In [1]:
%pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.32.0 trl==0.4.7
%pip install -U datasets -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.32.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0,

In [2]:
import os, torch, logging
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, IA3Config
from trl import SFTTrainer

2024-05-07 13:39:25.614179: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 13:39:25.614311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 13:39:25.748775: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Converting user message to saiga prompt style
system_message = "Ты — доцент, кандидат наук, эксперт, который осуществляет допуск квалификационных работ к защите'. Тебе нужно провести предварительную экспертизу основных структурных элементов научно-исследовательской работы: цель, задачи и заключения на наличие, взаимосвязь и логичность."


def conversation_prep(data, sys_prompt=system_message):
    user_prompt = data["user"]
    outp = data["output"]
    system_prompt = ""
    if sys_prompt != "":
        system_prompt = f"<s>system\n{system_message}</s>\n"
    prompt = f""
    return {"text": f"{system_prompt}<s>user\n{user_prompt}</s>\n<s>bot\n{outp}</s>\n"}

In [4]:
import csv

# Parsing dataset
data = {"user": [], "output": []}
with open("/kaggle/input/vkr-dataset/train.csv", "r", newline="") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=",")
    next(spamreader)
    for row in spamreader:
        user = f"""Вступление: {row[1].strip()}\nЗаключение: {row[2].strip()}\nСкажи что есть в вступлении и чего не хватает в заключении."""
        output = f"Ответ: {row[3].strip()}"
        data["user"].append(user)
        data["output"].append(output)

In [5]:
from datasets import Dataset, Features, Value

# Create dataset class
dataset = Dataset.from_dict(data)
dataset = dataset.map(conversation_prep)
dataset = dataset.remove_columns(["user", "output"])
print(dataset[0]["text"])

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

<s>system
Ты — доцент, кандидат наук, эксперт, который осуществляет допуск квалификационных работ к защите'. Тебе нужно провести предварительную экспертизу основных структурных элементов научно-исследовательской работы: цель, задачи и заключения на наличие, взаимосвязь и логичность.</s>
<s>user
Вступление: Теория контрактов представляет собой раздел теоретической экономики. Она исследует определение параметров контракта между заключающими его сторонами. Данная теория возникла в начале двадцатого века и бурно развивалась, в связи с чем возникли многие экономические труды таких ученых, как Р. Коуз, К. Янковский, В. Лисин и многих других [10] В разные года многие ученые получали нобелевские премии за развитие данной сферы. В 2016-ом году этого звания удостоились ученые О. Харт и Б. Хольфстрем. за труды по исследованию классической модели «принципал-агент», которая в данной работе формулируется отношениями между фирмой, которая сталкивается с проблемой стимуляции сотрудников, и агентом, чь

In [6]:
# Model and tokenizer names
base_model_name = "pieken/saiga2_7b_lora_merged"
refined_model = "Megnis/saiga2-7b-vkr-analis"

# Load tokenizer and set it's settings
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_8bit=True, bnb_8bit_quant_type="nf4", bnb_8bit_compute_dtype=torch.float16, bnb_8bit_use_double_quant=False
)

# Load model and set it's settings
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, quantization_config=quant_config, device_map={"": 0})
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
# Setting IA3 Config
peft_parameters = IA3Config(task_type="CAUSAL_LM", inference_mode=False)
# Setting Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    # evaluation_strategy="steps",#Not helpful because we donot want to evaluate the model we just want to train it
    # eval_steps=1000,
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="linear",
    warmup_steps=10,
    report_to="tensorboard",
)

# Create Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params,
)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/345 [00:00<?, ? examples/s]

In [8]:
# Create Trainer
fine_tuning.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to fl

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
# Save Model
fine_tuning.model.save_pretrained(refined_model)

In [ ]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")

from huggingface_hub import login

login(secret_value_0)

fine_tuning.push_to_hub(refined_model)

In [ ]:
# Load IA3 model
ft_model = PeftModel.from_pretrained(base_model, refined_model)

In [ ]:
system_message = "Ты — доцент, кандидат наук, эксперт, который осуществляет допуск квалификационных работ к защите'. Тебе нужно провести предварительную экспертизу основных структурных элементов научно-исследовательской работы: цель, задачи и заключения на наличие, взаимосвязь и логичность."


def prompt_prep(data, sys_prompt=system_message):
    user_prompt = data["user"]
    system_prompt = ""
    if sys_prompt != "":
        system_prompt = f"<s>system\n{system_message}</s>\n"
    prompt = f""
    return {"text": f"{system_prompt}<s>user\n{user_prompt}</s>\n<s>bot\n"}

In [ ]:
data_test = {"user": []}
raw_data_test = {"context_start": [], "context_end": [], "answer": []}
with open("/kaggle/input/vkr-dataset/test.csv", "r", newline="") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=",")
    next(spamreader)
    for row in spamreader:
        raw_data_test["context_start"].append(row[1].strip())
        raw_data_test["context_end"].append(row[2].strip())
        raw_data_test["answer"].append(row[3].strip())
        user = f"""Вступление: {row[1].strip()}\nЗаключение: {row[2].strip()}\nСкажи что есть в вступлении и чего не хватает в заключении."""
        data_test["user"].append(user)

In [ ]:
prompts = Dataset.from_dict(data_test)
prompts = prompts.map(prompt_prep)
prompts = prompts.remove_columns(["user"])
# print(prompts[0]["text"])
prompts

In [ ]:
# function for answers generating
def generate(model, prompt: str) -> str:
    device = "cuda"  # the device to load the model onto
    model_inputs = llama_tokenizer([prompt], return_tensors="pt").to(device)
    generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
    generated = llama_tokenizer.batch_decode(generated_ids)
    return generated[0][len(prompt) :].strip()

In [ ]:
# saving results in dataset
result_dataset = "Megnis/results_vkr"
data_result = {"question": [], "context_start": [], "context_end": [], "answer": [], "model_answer": []}
for idx, prompt in enumerate([prompts[5]]):
    print(prompt)
    result = generate(ft_model, prompt["text"])
    data_result["question"].append("")
    data_result["context_start"].append(raw_data_test["context_start"][idx])
    data_result["context_end"].append(raw_data_test["context_end"][idx])
    data_result["answer"].append(raw_data_test["answer"][idx])
    data_result["model_answer"].append(result)
    print(result)

    results = Dataset.from_dict(data_result)
    results.push_to_hub(result_dataset)